In [1]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.5,
    groq_api_key="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"
)

In [2]:
response = llm.invoke("who is elon musk?")
print(response.content)

Elon Musk is a South African-born entrepreneur, inventor, and business magnate. He is best known for his ambitious goals and innovative ventures in the fields of electric cars, space exploration, renewable energy, and artificial intelligence.

Here are some of his notable achievements and companies:

1. **PayPal**: Musk co-founded PayPal, an online payment system, in 1998. He sold the company to eBay for $1.5 billion in 2002.
2. **SpaceX**: In 2002, Musk founded SpaceX, a private aerospace manufacturer and space transport services company. SpaceX has successfully launched numerous rockets, including the Falcon 9 and Falcon Heavy, and has become a leading player in the private space industry.
3. **Tesla, Inc.**: In 2004, Musk co-founded Tesla, Inc., which has become one of the leading electric vehicle manufacturers in the world. Tesla's mission is to accelerate the world's transition to sustainable energy through its electric cars, solar power systems, and energy storage products.
4. **

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://careers.airbnb.com/positions/6580100/')
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.

















Backend Staff Software Engineer, On Trip - Careers at Airbnb







































Skip to main content


		Careers	  

Life at Airbnb
Job Search

Connect Engineering Apprenticeship
Internship Programs
Contractor Roles


 

















Life at Airbnb
Job Search

Connect Engineering Apprenticeship
Internship Programs
Contractor Roles


 





Backend Staff Software Engineer, On Trip


United States




Role overview
Application



Airbnb was born in 2007 when two Hosts welcomed three guests to their San Francisco home, and has since grown to over 4 million Hosts who have welcomed more than 1 billion guest arrivals in almost every country across the globe. Every day, Hosts offer unique stays and experiences that make it possible for guests to connect with communities in a more authentic way.

The Community You Will Join: 

Airbnb’s On Trip and Wishlist teams are responsible for providing Guests a seamless journey from trip planning to trip end.
Product

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    SCRAPED DATA is given below:
    {page_data}
    QUESTION is given below:
    The following is a job description from Airbnb. I want you to extract the job postings from the page and return them in JSON format.
    The keys could be job_title, job_description, job_location, job_type, skills_required.
    Only return the JSON object, nothing else.
"""
)
chain = prompt_template | llm

reply = chain.invoke(input={"page_data": page_data})
print(reply.content)

```
{
  "job_title": "Backend Staff Software Engineer, On Trip",
  "job_description": "Explore, shape, and develop new product experiences alongside cross-functional partners (design and product); from ideation to implementation at scale. Develop prototypes to validate ideas while proactively considering long-term implications on technical design and architecture. Translate prototypes to high quality, scalable, and maintainable production code through iterative development. Cross-collaborate across engineering to build new features holistically.",
  "job_location": "United States",
  "job_type": "Full-time",
  "skills_required": [
    "9+ years of experience in a hands-on software engineering role in backend development",
    "Bachelor's and/or Master/PhD's degree, preferably in CS, or equivalent experience",
    "Extensive experience working in Java and/or Kotlin",
    "Passionate about efficiency, availability, technical quality and system quality",
    "Passionate about design and U

In [29]:
type(reply.content)

str

In [23]:

from langchain_core.output_parsers import JsonOutputParser

output_parser = JsonOutputParser()
job = output_parser.parse(reply.content)
job

{'job_title': 'Backend Staff Software Engineer, On Trip',
 'job_description': 'Explore, shape, and develop new product experiences alongside cross-functional partners (design and product); from ideation to implementation at scale. Develop prototypes to validate ideas while proactively considering long-term implications on technical design and architecture. Translate prototypes to high quality, scalable, and maintainable production code through iterative development. Cross-collaborate across engineering to build new features holistically.',
 'job_location': 'United States',
 'job_type': 'Full-time',
 'skills_required': ['9+ years of experience in a hands-on software engineering role in backend development',
  "Bachelor's and/or Master/PhD's degree, preferably in CS, or equivalent experience",
  'Extensive experience working in Java and/or Kotlin',
  'Passionate about efficiency, availability, technical quality and system quality',
  'Passionate about design and UX sensibilities',
  'Exc

In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [7]:
import pandas as pd

data = {
    'Tech Stack': [
        'Python, Django, Flask',
        'JavaScript, React, Node.js',
        'HTML, CSS, Bootstrap',
        'Java, Spring Boot, Hibernate',
        'Ruby on Rails, PostgreSQL',
        'C++, Qt, OpenGL',
        'Swift, iOS Development, Objective-C',
        'PHP, Laravel, MySQL',
        'TypeScript, Angular, Node.js',
        'Go, Kubernetes, Docker'
    ],
    'Portfolio Website': [
        'https://pythonportfolio.com',
        'https://jsreactdev.com',
        'https://webdevportfolio.com',
        'https://javadevportfolio.com',
        'https://rubyrailsportfolio.com',
        'https://cppqtportfolio.com',
        'https://swiftiosdev.com',
        'https://phpdevportfolio.com',
        'https://typescriptangular.com',
        'https://gok8sportfolio.com'
    ]
}

df = pd.DataFrame(data)

In [8]:
csv_file_path = 'tech_stack.csv'
df.to_csv(csv_file_path, index=False)

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [14]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# Initialize HuggingFaceEmbeddings with the correct model name
embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [20]:
import chromadb

client = chromadb.Client()
collection = client.get_or_create_collection("tech_stack_portfolio")

for index, row in df.iterrows():
    tech_stack = row["Tech Stack"]
    url = row["Portfolio Website"]
    
    # Generate embeddings for Tech Stack and URL using embed_documents
    embeddings_list = embeddings.embed_documents([tech_stack, url])
    tech_stack_embedding = embeddings_list[0]
    url_embedding = embeddings_list[1]
    
    # Insert into ChromaDB collection
    collection.add(
        documents=[tech_stack, url],
        embeddings=[tech_stack_embedding, url_embedding],
        metadatas=[{"tech_stack": tech_stack, "url": url}, {"tech_stack": tech_stack, "url": url}],
        ids=[f"{index}_tech_stack", f"{index}_url"]
    )

In [21]:
# for the error that came earlier

# The error you're encountering suggests that the HuggingFaceEmbeddings object in LangChain does not have an embed method. This is a common issue, and the reason is that the HuggingFaceEmbeddings class in LangChain works slightly differently from other embedding classes.

# Correct Way to Use HuggingFaceEmbeddings
# Instead of calling the embed() method directly, you need to use the embed_query or embed_documents method depending on what you're trying to do. For embedding documents (like tech stacks and URLs), you should use embed_documents.

# Here’s how you can correctly generate embeddings:

# Step 1: Fix the embed Method Issue
# You should use the embed_documents method provided by the HuggingFaceEmbeddings class. Here's how you can update your code:

In [22]:
query = "Python, Django"
query_embedding = embeddings.embed_documents([query])[0]

# Retrieve the most similar documents from ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5  # Number of top results to return
)

# Output the results
for result in results['documents']:
    print(f"Tech Stack: {result[0]}, Portfolio URL: {result[1]}")

Tech Stack: Python, Django, Flask, Portfolio URL: https://pythonportfolio.com


In [25]:
link_list = collection.query(query_texts=job['skills_required'], n_results=2).get('metadatas', [])

In [30]:
prompt_email = PromptTemplate.from_template(
            """
            ### JOB DESCRIPTION:
            {job_description}

            ### INSTRUCTION:
            You are Harsha, a business development executive at Infosys. Infosys is an AI & Software Consulting company dedicated to facilitating
            the seamless integration of business processes through automated tools. 
            Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
            process optimization, cost reduction, and heightened overall efficiency. 
            Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Infosys 
            in fulfilling their needs.
            Also add the most relevant ones from the following links to showcase Infosys's portfolio: {link_list}
            Remember you are Harsha, BDE at Infosys. 
            Do not provide a preamble.
            ### EMAIL (NO PREAMBLE):

            """
        )

In [31]:
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": job['job_description'], "link_list": link_list})
print(res.content)

Subject: Unlocking New Product Experiences with Infosys

Dear Hiring Manager,

I came across your job description for exploring, shaping, and developing new product experiences, and I'm excited to introduce Infosys as a potential partner to help you achieve your goals. At Infosys, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team of experts has extensive experience in developing prototypes and translating them into high-quality, scalable, and maintainable production code. We have a strong portfolio of delivering successful projects using a range of technologies, including:

* JavaScript, React, and Node.js, as seen in our work at https://jsreactdev.com
* Java, Spring Boot, and Hibernate, showcased at https://javadevportfolio.com
* HTML, CSS, and Bootstrap, demonstrated at https://webdevportfolio.com

We understand the importance of cross-functional c